<a href="https://colab.research.google.com/github/ShreyaDesai2/py/blob/master/Breast_Detection_AJ1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The breast cancer database is a publicly available dataset from the UCI Machine learning Repository. The downloaded data set is .data file. The file extention can be changed to .csv file. The columns are named as ‘id’, ‘clump thickness’, ‘uniformity of cell size’, ‘uniformity of cell shape’, ‘marginal adhesion’, ‘single epithelial cell size’, ‘bare nuclei’, ‘bland chromatin’, ‘normal nucleoli’, ‘mitosis’ and ‘class’. This gives information on tumor features such as tumor size, density, and texture.
The goal is then to create a classification model (mainly Support vector Classification) that looks at predicts if the cancer diagnosis is benign or malignant based on several features.

In [ ]:
# Step -1 - Import Package
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import seaborn as sns
from sklearn import metrics
plt.rcParams["figure.figsize"] = (10, 10)


Before applying any method, we need to check if any values are missing and then deal with them if so. In this dataset, there are no missing values. Then Data is divided into the Train set and Test set. We use the Train set to make the algorithm learn the data’s behavior and then check the accuracy of our model on the Test set.
Features (X): The columns that are inserted into our model will be used to make predictions.
Prediction (y): Target variable that will be predicted by the features.


In [ ]:
# Step - 2 - Define the main function
def main():
    # Get data
    cancer_data = load_breast_cancer()
    cancer_data_X = pd.DataFrame(cancer_data.data, columns = cancer_data.feature_names)
    cancer_data_y = cancer_data.target
    features = cancer_data.feature_names
    
    vars = ['mean radius', 'mean texture', 'mean area', 'mean perimeter', 'mean smoothness']
    ## Data Exploration
    print(f'The features in dataset are: {features}')
    #print(f'Data description\n {cancer_data_X.describe()}')
    
    #Plots
    plot_data(cancer_data_X, cancer_data_y, features= vars, cor=True)

    ## Remove Outliers
    cancer_data_X, cancer_data_y = remove_outliers(cancer_data_X,cancer_data_y, features)
    
    X_train, y_train, X_test, y_test = preprocess(cancer_data_X, cancer_data_y, features)
    model = SVC(random_state=6)

    model = train(model, X_train, y_train)
    
    baseline = evaluate(model, X_test, y_test, bl=True)

    best_params = optimize_models(X_train, y_train)
    print(best_params)

    ## Build Best Model
    best_C= best_params['C']
    best_kernel = best_params['kernel']

    best_model = SVC(kernel = best_kernel, C= best_C, random_state=6)
    best_model = train(best_model, X_train, y_train)
    evaluate (best_model, X_test, y_test)


Using the confusion matrix will give us important information: 
1. We had 171 women in our test set
2. 4 women were classified as not having when actually they had (Type 1 error).
3. 1 were classified as having breast cancer whey they did not (Type 2 error).

In [ ]:
# Step - 3 - Plot graphs to understand data
def plot_data(x_df, y_df,features, cor=False):
    X = x_df.copy(deep=True)
    X['class'] = y_df
    sns.pairplot(X, hue = 'class', vars = ['mean radius', 'mean texture', 'mean area', 'mean perimeter', 'mean smoothness'] )
    plt.show()
    
    if cor:
      corr = X[features].corr()
      plt.figure(figsize=(10,10))
      sns.heatmap(corr, cbar=True, square= True, fmt='.1f', annot=True, annot_kws={'size':15}, cmap='Greens')
      plt.show()


So what can we do to improve our model ? 
First, let's remove the outliers from the database using the inplace function. secondly, we normalize the data set. Using the Feature scaling will help us to see all the variables from the same lens (same scale), in this way we will bring all values into the range [0,1]:

In [ ]:
# Step - 4 - Preprocess data
# Step -4a : Remove outliers
def remove_outliers(x,y, features):
    #remove null
    x_df = x.copy(deep=True)
    x_df['class'] = y
    x_df.dropna(inplace=True)
    return x_df[features], x_df['class']

# Step -4b : Normalize data
def scale_numeric(df):
    x = df.values 
    scaler = preprocessing.StandardScaler()
    x_scaled = scaler.fit_transform(x)
    df = pd.DataFrame(x_scaled)
    return df
# Step -4b : Preprocess data
def preprocess(x, y, features):
    x_df = x[features].copy(deep=True)
    x_df = scale_numeric(x_df)
    #print(len(x_df),len(y))
    # Split data into train, test
    X_train, X_test, y_train, y_test = train_test_split(x_df,y, test_size=0.3, random_state=45)
    return X_train, y_train, X_test, y_test
  


So after training the model, what will the SVC model actually do? 
It’s important to start with the intuition for SVM with the special linearly separable classification case. If classification of observations is “linearly separable”, SVM fits the “decision boundary” that is defined by the largest margin between the closest points for each class. This is commonly called the “maximum margin hyperplane (MMH)”.

Note that: In the training phase, the learning algorithm uses the training data to adjust the model’s parameters to minimize errors. At the end of the training phase, you get the trained model.
In the testing phase, the trained model is applied to test data. Test data is separate from the training data, and is previously unseen by the model. The model is then evaluated on how it performs on the test data. The goal in building a classifier model is to have the model perform well on training as well as test data.

In [ ]:
# Step - 5 - train model 
def train(model,X_train, y_train):
    model.fit(X_train, y_train)
    return model

as previously mentioned, we wanted to make sure that our classifier is efficiently categorizing the dataset in hand. To do it, we need performance metrics. Here we are as metrics: 
1. Confusion matrix: which specified the negatif and positive truly classified features.
2. The accuracy or precision which specifies how many times the prediction is right. 

In [ ]:
# Step - 6 - Evaluate Model
def evaluate(model, X_test, y_test, plot = True, print_results=True, bl=False):
    y_pred = model.predict(X_test)
    cm = metrics.confusion_matrix(y_test, y_pred)
    acc = metrics.accuracy_score(y_test, y_pred)
    if print_results:
      if bl:
        print('\n\nBaseline Model Performance on Test Dataset:\n')
      else:
        print('\n\nBest Model Performance on Test Dataset:\n')
      print('\nConfusion Matrix:\n',cm)
      print(f'Accuracy: {acc*100}%')

    if plot:
      sns.heatmap(cm, annot= True)
      plt.show()
    return 



So now as we trained and tested our model, let's try to enhance it's accuracy. 
We will create a similar function to the function above used to cross validate and optimize the hyperparamters of the RBF kernel SVC and C. we noticed that we receive a 98% accuracy using the optimized hyperparameters C=5 along with the radial basis function.

Conclusion: As you have seen SVMs easily get above 90% on this dataset. With just a bit of hyperparameter tuning we can push this number up to a remarkable 97%. Moreover, SVMs are a very simple algorithm and have taken no more than a few seconds to train on the computer. The potential to create models so easily that are this accurate on a task as complicated as breast cancer classification is perhaps one of the reasons that machine learning has become such a hot topic as of late.

In [ ]:
# Step - 7 - Improve Model
def optimize_models(X_train, y_train):
  params = {'kernel':['rbf'], 'C':[1.0, 5.0, 10]}
  model = SVC(random_state=5)
  clf = GridSearchCV(model, params)
  clf.fit(X_train, y_train)
  return clf.best_params_


# call the main finction
if __name__ == '__main__':
    main()